In [25]:
import numpy as np
import pandas as pd
#import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
# from statsmodels.stats.proportion import proportions_ztest
# from statsmodels.stats.proportion import proportions_chisquare
from IPython.display import display, HTML
from scipy.stats import kruskal

import sys
import logging
logging.basicConfig(level=logging.CRITICAL)

In [26]:
def two_proprotions_test(success_a, size_a, success_b, size_b):
    prop_a = success_a / size_a
    prop_b = success_b / size_b
    prop_pooled = (success_a + success_b) / (size_a + size_b)
    var = prop_pooled * (1 - prop_pooled) * (1 / size_a + 1 / size_b)
    zscore = np.abs(prop_b - prop_a) / np.sqrt(var)
    one_side = 1 - stats.norm(loc = 0, scale = 1).cdf(zscore)
    pvalue = one_side * 2
    return zscore, pvalue

In [27]:
all_data = pd.read_csv('/home/tiger/archived-data/aeolus-data/20201228/10/34979364-课程实验-掌握度模型实验-查询20.csv', encoding = 'gb18030')
# all_data.head()

In [30]:
### 预处理数据
#1. 填充空值数据
#2. 增加新字段

all_data = all_data[all_data.answers_kp_video_review >0]

review_test_data = all_data

review_test_data['fluctuate_kp_ratio'] = review_test_data['answers_fluctuate_kp_video_review'] / review_test_data['answers_kp_video_review']

# review_test_data.head()


#### 全量用户 

In [33]:
def ab_summary_data(data):
    test_summary = pd.DataFrame(columns = ["User_Group", "Metrics", "Control", "Test1", "Test2", "Delta", "p_value","Significance"])
    
    test1_data = data.loc[data.user_group == '掌握度实验组']
    test2_data = data.loc[data.user_group == '推荐实验组']
    ctrl_data = data.loc[data.user_group == '对照组']

    dnu_t1 = len(test1_data)
    dnu_t2 = len(test2_data)
    dnu_c = len(ctrl_data)

    # 新增用户
    test_summary = test_summary.append({"Metrics": '实验用户数', "Control": '{:,}'.format(dnu_c), "Test1": '{:,}'.format(dnu_t1), "Test2": '{:,}'.format(dnu_t2), "Delta": '-', "p_value": '-', "Significance": '-'}, ignore_index = True)
    
    # 掌握度异常波动知识点占比
    fl_kp_ratio_t1 = test1_data.fluctuate_kp_ratio.mean()
    fl_kp_ratio_t2 = test2_data.fluctuate_kp_ratio.mean()
    fl_kp_ratio_c = ctrl_data.fluctuate_kp_ratio.mean()
    
    fl_kp_ratio_delta = fl_kp_ratio_t1 / fl_kp_ratio_c - 1 if fl_kp_ratio_c >0 else np.nan
    fl_kp_ratio_p_value = stats.f_oneway(test1_data.fluctuate_kp_ratio, test2_data.fluctuate_kp_ratio, ctrl_data.fluctuate_kp_ratio).pvalue
    
    fl_kp_ratio_sig =  "Significant" if fl_kp_ratio_p_value < 0.1 else "Non-significant"
    test_summary = test_summary.append({"Metrics": '掌握度异常波动知识点占比', "Control": "{:.2%}".format(fl_kp_ratio_c), "Test1": "{:.2%}".format(fl_kp_ratio_t1), "Test2": "{:.2%}".format(fl_kp_ratio_t2),  "Delta": "{:.2%}".format(fl_kp_ratio_delta), "p_value": round(fl_kp_ratio_p_value, 4), "Significance": fl_kp_ratio_sig}, ignore_index = True)

    # 复习课学习时长
    review_study_t1 = sum(test1_data.study_duration_video_review) / 60
    review_study_t2 = sum(test2_data.study_duration_video_review) / 60
    review_study_c = sum(ctrl_data.study_duration_video_review) / 60
    
    review_peru_t1 = round(review_study_t1 / dnu_t1 , 2)
    review_peru_t2 = round(review_study_t2 / dnu_t2 , 2)
    review_peru_c = round(review_study_c / dnu_c , 2)

    review_peru_delta = review_peru_t1 / review_peru_c - 1 if review_peru_c > 0 else np.nan
    review_peru_p_value = stats.f_oneway(test1_data.study_duration_video_review, test2_data.study_duration_video_review, ctrl_data.study_duration_video_review).pvalue
    review_peru_sig =  "Significant" if review_peru_p_value < 0.1 else "Non-significant"
    test_summary = test_summary.append({"Metrics": '复习课学习时长: 分钟', "Control": str(round(review_peru_c,2)), "Test1": str(round(review_peru_t1,2)), "Test2": str(round(review_peru_t2,2)), "Delta": "{:.2%}".format(review_peru_delta), "p_value": round(review_peru_p_value, 4), "Significance": review_peru_sig}, ignore_index = True)

    # 复习课完课
    review_fin_t1 = sum(test1_data.finish_lessons_video_review)
    review_fin_t2 = sum(test2_data.finish_lessons_video_review)
    review_fin_c = sum(ctrl_data.finish_lessons_video_review)
    
    review_fin_peru_t1 = round(review_fin_t1 / dnu_t1 , 2)
    review_fin_peru_t2 = round(review_fin_t2 / dnu_t2 , 2)
    review_fin_peru_c = round(review_fin_c / dnu_c , 2)

    review_fin_peru_delta = review_fin_peru_t1 / review_fin_peru_c - 1 if review_fin_peru_c > 0 else np.nan
    review_fin_peru_p_value = stats.f_oneway(test1_data.finish_lessons_video_review, test2_data.finish_lessons_video_review, ctrl_data.finish_lessons_video_review).pvalue
    review_fin_peru_sig =  "Significant" if review_fin_peru_p_value < 0.1 else "Non-significant"
    test_summary = test_summary.append({"Metrics": '复习课完课', "Control": str(round(review_fin_peru_c,2)), "Test1": str(round(review_fin_peru_t1,2)), "Test2": str(round(review_fin_peru_t2,2)), "Delta": "{:.2%}".format(review_fin_peru_delta), "p_value": round(review_fin_peru_p_value, 4), "Significance": review_fin_peru_sig}, ignore_index = True)

    # 复习课做题数
    review_ans_t1 = sum(test1_data.answers_exercise_video_review)
    review_ans_t2 = sum(test2_data.answers_exercise_video_review)
    review_ans_c = sum(ctrl_data.answers_exercise_video_review)
    
    review_ans_peru_t1 = round(review_ans_t1 / dnu_t1 , 2)
    review_ans_peru_t2 = round(review_ans_t2 / dnu_t2 , 2)
    review_ans_peru_c = round(review_ans_c / dnu_c , 2)

    review_ans_peru_delta = review_ans_peru_t1 / review_ans_peru_c - 1 if review_ans_peru_c > 0 else np.nan
    review_ans_peru_p_value = stats.f_oneway(test1_data.answers_exercise_video_review, test2_data.answers_exercise_video_review, ctrl_data.answers_exercise_video_review).pvalue
    review_ans_peru_sig =  "Significant" if review_ans_peru_p_value < 0.1 else "Non-significant"
    test_summary = test_summary.append({"Metrics": '复习课答题', "Control": str(round(review_ans_peru_c,2)), "Test1": str(round(review_ans_peru_t1,2)), "Test2": str(round(review_ans_peru_t2,2)), "Delta": "{:.2%}".format(review_ans_peru_delta), "p_value": round(review_ans_peru_p_value, 4), "Significance": review_ans_peru_sig}, ignore_index = True)

    test_summary['User_Group'] = '全量用户'
    
    return test_summary

In [34]:
output = ab_summary_data(review_test_data)
output.style.apply(lambda x: ["background: green" if v == 'Significant' else "" for v in x], axis = 1).hide_index()

User_Group,Metrics,Control,Test1,Test2,Delta,p_value,Significance
全量用户,实验用户数,"2,302","2,498","2,439",-,-,-
全量用户,掌握度异常波动知识点占比,8.32%,3.96%,8.55%,-52.38%,0.000000,Significant
全量用户,复习课学习时长: 分钟,9.26,9.3,9.07,0.43%,0.536000,Non-significant
全量用户,复习课完课,1.38,1.43,1.36,3.62%,0.002700,Significant
全量用户,复习课答题,32.42,33.72,32.68,4.01%,0.149800,Non-significant
